## Creación de la base de datos en SQL Server

### Importar la librería, asignar las variables y crear la conexión

In [19]:
import pyodbc

server = 'localhost\\SQLEXPRESS'
database = 'master'

# Create a connection string
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + server + ';'  # replace with your server name
    'DATABASE=' + database + ';'  # the database you want to connect to
    'Trusted_Connection=yes;'
)

### Crear la base de datos

In [20]:
# Create a connection
conn = pyodbc.connect(conn_str, autocommit=True) # El autocommint en True permite crear la base desde el código

# Create a cursor
cursor = conn.cursor()

# Execute the CREATE DATABASE statement
cursor.execute('CREATE DATABASE Ecommerce_Brasil')  # replace with your database name 

# Close the cursor and the connection
cursor.close()
conn.close()

### Creación de las tablas

In [21]:
database = 'Ecommerce_Brasil' # Cambiar a la base de datos que ocupemos

# Create a connection string
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + server + ';'  # replace with your server name
    'DATABASE=' + database + ';'  # the database you want to connect to
    'Trusted_Connection=yes;'
)

# Create a connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Crear un cursor
cursor = conn.cursor()

# Crear la tabla de Orders
cursor.execute("""
CREATE TABLE Orders (
    order_id VARCHAR(50) PRIMARY KEY,
    customer_id VARCHAR(50),
    order_status VARCHAR(20),
    order_purchase_timestamp DATETIME,
    order_approved_at DATETIME,
    order_delivered_carrier_date DATETIME,
    order_delivered_customer_date DATETIME,
    order_estimated_delivery_date DATETIME
)
""")

# Crear la tabla de Customers
cursor.execute("""
CREATE TABLE Customers (
    customer_id VARCHAR(50) PRIMARY KEY,
    customer_unique_id VARCHAR(50),
    customer_zip_code_prefix INT,
    customer_city VARCHAR(50),
    customer_state VARCHAR(10)
)
""")

# Crear la tabla de Items
cursor.execute("""
CREATE TABLE Items (
    item_id INT PRIMARY KEY,
    order_id VARCHAR(50),
    order_item_id INT,
    product_id VARCHAR(50),
    seller_id VARCHAR(50),
    shipping_limit_date DATETIME,
    price DECIMAL(10, 2),
    freight_value DECIMAL(10, 2)
)
""")

# Crear la tabla de Payments
cursor.execute("""
CREATE TABLE Payments (
    payment_id INT PRIMARY KEY,
    order_id VARCHAR(50),
    payment_sequential INT,
    payment_type VARCHAR(20),
    payment_installments INT,
    payment_value DECIMAL(10, 2)
)
""")

# Crear la tabla de Reviews
cursor.execute("""
CREATE TABLE Reviews (
    review_id VARCHAR(50) PRIMARY KEY,
    order_id VARCHAR(50),
    review_score INT,
    review_creation_date DATETIME,
    review_answer_timestamp DATETIME
)
""")

# Crear la tabla de Sellers
cursor.execute("""
CREATE TABLE Sellers (
    seller_id VARCHAR(50) PRIMARY KEY,
    seller_zip_code_prefix INT,
    seller_city VARCHAR(50),
    seller_state VARCHAR(10)
)
""")

# Crear la tabla de geolocation
cursor.execute("""
CREATE TABLE Geolocation (
    geolocation_id INT PRIMARY KEY,
    geolocation_zip_code_prefix INT,
    geolocation_lat DECIMAL(12, 10),
    geolocation_lng DECIMAL(12, 10),
    geolocation_city VARCHAR(50),
    geolocation_state VARCHAR(10)
)
""")

# Crear la tabla de Product_Categories
cursor.execute("""
CREATE TABLE Product_Categories (
    product_id VARCHAR(50) PRIMARY KEY,
    product_category_name_english VARCHAR(50)
)
""")



### Creación de las relaciones

In [22]:
# Crear la relación entre Orders y Customers
cursor.execute("""
ALTER TABLE Orders
ADD CONSTRAINT FK_Orders_Customers
FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
""")

# Crear la relación entre reviews y Orders
cursor.execute("""
ALTER TABLE Reviews
ADD CONSTRAINT FK_Reviews_Orders
FOREIGN KEY (order_id) REFERENCES Orders(order_id)
""")

# Crear la relacion entre payments y Orders
cursor.execute("""
ALTER TABLE Payments
ADD CONSTRAINT FK_Payments_Orders
FOREIGN KEY (order_id) REFERENCES Orders(order_id)
""")

# Crear la relación entre Items y Orders
cursor.execute("""
ALTER TABLE Items
ADD CONSTRAINT FK_Items_Orders
FOREIGN KEY (order_id) REFERENCES Orders(order_id)
""")

# Crear la relación entre Items y product_categories
cursor.execute("""
ALTER TABLE Items
ADD CONSTRAINT FK_Items_Product_Categories
FOREIGN KEY (product_id) REFERENCES Product_Categories(product_id)
""")

# Crear la relación entre Items y Sellers
cursor.execute("""
ALTER TABLE Items
ADD CONSTRAINT FK_Items_Sellers
FOREIGN KEY (seller_id) REFERENCES Sellers(seller_id)
""")

# Crear la relación entre Sellers y geolocation
cursor.execute("""
ALTER TABLE Sellers
ADD CONSTRAINT FK_Sellers_Geolocation
FOREIGN KEY (seller_zip_code_prefix) REFERENCES Geolocation(geolocation_zip_code_prefix)
""")

# Crear la relación entre Customers y geolocation
cursor.execute("""
ALTER TABLE Customers
ADD CONSTRAINT FK_Customers_Geolocation
FOREIGN KEY (customer_zip_code_prefix) REFERENCES Geolocation(geolocation_zip_code_prefix)
""")

# Confirmar los cambios
conn.commit()

# Cerrar la conexión
cursor.close()
conn.close()

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There are no primary or candidate keys in the referenced table 'Geolocation' that match the referencing column list in the foreign key 'FK_Sellers_Geolocation'. (1776) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")